In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\jsb24\\anaconda3\\envs\\PySpark\\Lib\\site-packages\\pyspark'

In [52]:
import pandas as pd
import numpy as np

In [9]:
!pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     ---- -------------------------------- 30.7/250.0 kB 325.1 kB/s eta 0:00:01
     --------- --------------------------- 61.4/250.0 kB 363.1 kB/s eta 0:00:01
     ------------- ----------------------- 92.2/250.0 kB 476.3 kB/s eta 0:00:01
     ---------------------------- ------- 194.6/250.0 kB 784.3 kB/s eta 0:00:01
     ------------------------------------ 250.0/250.0 kB 901.3 kB/s eta 0:00:00


С помощью модуля pandas преобразуйте файл из .xlsx в .csv формат:

In [3]:
raw_data = pd.read_excel('online_retail.xlsx')


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
raw_data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


С помощью модуля pandas преобразуйте файл из .xlsx в .csv формат

In [6]:
raw_data.to_csv('online_retail.csv', index=False)

Инициализируйте Spark-сессию

In [67]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [8]:
# Create SparkSession 
spark = SparkSession.builder \
  .master("local[1]") \
  .appName("SparkFirst") \
  .config("spark.executor.memory", "10g")\
  .config("spark.executor.cores", 5) \
  .config("spark.dynamicAllocation.enabled", "true") \
  .config("spark.dynamicAllocation.maxExecutors", 5) \
  .config("spark.shuffle.service.enabled", "true") \
  .getOrCreate()

Создайте dataframe из скачанного файла:

In [45]:
df = spark.read.csv('online_retail.csv', header=True)

In [46]:
df.show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



Подсчитайте количество строк в файле:

In [11]:
lines_cnt = df.count()
print('Количество строк:', lines_cnt)

Количество строк: 541909


Подсчитайте количество уникальных клиентов:

In [12]:
num_cust = df.select("CustomerID").distinct().count()
print('Количество уникальных клиентов:', num_cust)

Количество уникальных клиентов: 4373


Подсчитайте в какой стране совершается большинство покупок

In [13]:
country_max_sales = df.groupBy('Country').count().orderBy('count', ascending=False).first()[0]
print('Страна с наибольшим количеством покупок:', country_max_sales)

Страна с наибольшим количеством покупок: United Kingdom


Подсчитайте дату самой ранней покупки

In [15]:
min_date = df.select(min('InvoiceDate')).first()[0]
print('Самая ранняя дата покупки:', min_date)

Самая ранняя дата покупки: 2010-12-01 08:26:00


Подсчитайте дату самой последней покупки на платформе

In [16]:
max_date = df.select(max('InvoiceDate')).first()[0]
print("Самая поздняя дата покупки:", max_date)

Самая поздняя дата покупки: 2011-12-09 12:50:00


__Проведите RFM-анализ клиентов платформы.__ 

Добавьте в dataframe для каждого клиента 3 новых поля - recency (как давно он покупал), frequency (частота), monetary_value(денежная ценность).

Денежная ценность клиента - это сумма всех его покупок:

In [47]:
df_mon=df.withColumn('Total_sum_order',round (col('UnitPrice')*col('Quantity'),2))
df_mon = df_mon.groupBy('CustomerID').agg(sum('Total_sum_order').alias('Monetary_value'))
df_mon.show(5)

+----------+-----------------+
|CustomerID|   Monetary_value|
+----------+-----------------+
|   15039.0|19786.43999999997|
|   16553.0|5664.569999999999|
|   13178.0|5725.470000000003|
|   17786.0|           278.74|
|   12891.0|            331.0|
+----------+-----------------+
only showing top 5 rows



Частота покупок - это число чеков клиента:

In [48]:
df_freq = df.groupBy('CustomerID').agg(count('InvoiceNo').alias('Frequency'))
df_freq.show(5)

+----------+---------+
|CustomerID|Frequency|
+----------+---------+
|   15039.0|     1508|
|   16553.0|       86|
|   13178.0|      265|
|   17786.0|       72|
|   12891.0|        3|
+----------+---------+
only showing top 5 rows



Давность - это разница текущего момента и последней даты покупки клиента.

In [39]:
today = current_date()

In [49]:
df_rec = df.groupBy('CustomerID').agg(max('InvoiceDate').alias('Max_date'))
df_rec = df_rec.withColumn('Recency', datediff(today, df_rec['Max_date']))
df_rec.show(5)

+----------+-------------------+-------+
|CustomerID|           Max_date|Recency|
+----------+-------------------+-------+
|      null|2011-12-09 10:26:00|   4281|
|   12346.0|2011-01-18 10:17:00|   4606|
|   12347.0|2011-12-07 15:52:00|   4283|
|   12348.0|2011-09-25 13:13:00|   4356|
|   12349.0|2011-11-21 09:51:00|   4299|
+----------+-------------------+-------+
only showing top 5 rows



In [50]:
df_new = df.join(df_mon, 'CustomerID', how = 'inner').join(df_freq, 'CustomerID', how = 'inner').join(df_rec, 'CustomerID', how = 'inner')
df_new.show(5)

+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-----------------+---------+-------------------+-------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|       Country|   Monetary_value|Frequency|           Max_date|Recency|
+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-----------------+---------+-------------------+-------+
|   17850.0|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|United Kingdom|5288.630000000009|      312|2011-02-10 14:38:00|   4583|
|   17850.0|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|United Kingdom|5288.630000000009|      312|2011-02-10 14:38:00|   4583|
|   17850.0|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|United Kingdom|5288.630000000009|      312|2011-02-10 14:38:00|   4583|
|   17850.0|   5

Для каждого показателя добавьте стобец с разбиением клиентов на 3 группы. Допустим, у нас есть 3 клиента, первый клиент последний раз купил товар только в прошлом году, второй клиент в прошлом месяце, а третий клиент на прошлой неделе. Каждый из этих клиентов должен получить различные значения группы для показателя Recency - A, B и С, где А - отражает наибольшую “ценность”, а С - соответственно, наименьшую. 

In [61]:
quantiles_r = df_new.approxQuantile('Recency', [0.33, 0.66], 0.01)
quantiles_f = df_new.approxQuantile('Frequency', [0.33, 0.66], 0.01)
quantiles_m = df_new.approxQuantile('Monetary_value', [0.33, 0.66], 0.01)

In [62]:
df_new = df_new.withColumn('Recency_factor', when(df_new['Recency'] <= quantiles_r[0], 'C')
                           .when((df_new['Recency'] > quantiles_r[0]) & (df_new['Recency'] <= quantiles_r[1]), 'B')
                           .otherwise('A'))

In [63]:
df_new = df_new.withColumn('Frequency_factor', when(df_new['Frequency'] <= quantiles_f[0], 'C')
                           .when((df_new['Frequency'] > quantiles_f[0]) & (df_new['Frequency'] <= quantiles_f[1]), 'B')
                           .otherwise('A'))

In [64]:
df_new = df_new.withColumn('Monetary_factor', when(df_new['Monetary_value'] <= quantiles_m[0], 'C')
                           .when((df_new['Monetary_value'] > quantiles_m[0]) & (df_new['Monetary_value'] <= quantiles_m[1]), 'B')
                           .otherwise('A'))

In [70]:
rfm_table = df_new.withColumn('groups', F.concat(F.col('Recency_factor'),F.col('Frequency_factor'),F.col('Monetary_factor')))
rfm_table.select('CustomerID','Recency','Frequency','Monetary_value','Recency_factor','Frequency_factor','Monetary_factor', 'groups').show(5)

+----------+-------+---------+-----------------+--------------+----------------+---------------+------+
|CustomerID|Recency|Frequency|   Monetary_value|Recency_factor|Frequency_factor|Monetary_factor|groups|
+----------+-------+---------+-----------------+--------------+----------------+---------------+------+
|   15039.0|   4290|     1508|19786.43999999997|             B|               A|              A|   BAA|
|   15039.0|   4290|     1508|19786.43999999997|             B|               A|              A|   BAA|
|   15039.0|   4290|     1508|19786.43999999997|             B|               A|              A|   BAA|
|   15039.0|   4290|     1508|19786.43999999997|             B|               A|              A|   BAA|
|   15039.0|   4290|     1508|19786.43999999997|             B|               A|              A|   BAA|
+----------+-------+---------+-----------------+--------------+----------------+---------------+------+
only showing top 5 rows



Добавьте итоговый столбец с “суммой” значений групп по каждому показателю и сохраните в отдельный csv-файл Id только тех клиентов, у которых значения групп ААА.

In [74]:
best_clients = rfm_table.select(['CustomerID']).filter(rfm_table.groups == 'AAA').distinct()
best_clients.show(5)

+----------+
|CustomerID|
+----------+
|   17865.0|
|   17509.0|
|   12472.0|
|   15719.0|
|   16843.0|
+----------+
only showing top 5 rows



In [76]:
best_clients.toPandas().to_csv('best_clients.csv', index=False)